In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

データの読み込み

In [ ]:
df_sub = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv")
df_train = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv")
df_test = pd.read_csv("/kaggle/input/house-prices-advanced-regression-techniques/test.csv")

データの中身を一部確認

In [ ]:
df_train.head()

データの情報をdf.info()で確認できる

In [ ]:
df_train.info()

isnull()：データが欠損値（空の場合）True判定<br>
sum()：Trueの数を数える

In [ ]:
df_train.isnull().sum().head(100)

df.describe()：各列の特徴がわかる，count（データの数）, mean（平均）, std（標準偏差）など

In [ ]:
df_train.describe()

In [ ]:
missing = df_train.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar()

In [ ]:
df_train.isnull().sum()

GrLivAreaをx軸、SalePriceをy軸としてプロット<br>
正の相関が見える、外れ値も確認できる<br>
ちなみにGrLivAreaとはリビングの広さらしい、何故リビングの広さに着目したのか<br>
直観か？

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = df_train["GrLivArea"], y = df_train["SalePrice"])
plt.ylabel("SalePrice", fontsize=13)
plt.xlabel("GrLivArea", fontsize=13)
plt.show()

seabornのdistplotメソッドでヒストグラムを作成<br>
引数はデータフレーム型

In [ ]:
ax = sns.distplot(df_train["SalePrice"])

In [ ]:
ax = sns.boxplot(x = df_train["SalePrice"])

外れ値の除去<br>
リビングが広すぎるおよび家賃が30万未満のデータは外れ値であると判定

In [ ]:
df_train = df_train.drop(df_train[(df_train["GrLivArea"]>4000) & (df_train["SalePrice"]<300000)].index)

外れ値が消えたことを確認

In [ ]:
fig, ax = plt.subplots()
ax.scatter(df_train["GrLivArea"], df_train["SalePrice"])
plt.ylabel("SalePrice", fontsize=13)
plt.xlabel("GrLivArea", fontsize=13)
plt.show()

In [ ]:
ax = sns.boxplot(x=df_train["SalePrice"])

In [ ]:
ax = sns.distplot(df_train["SalePrice"])

ntrain：訓練データの形状

In [ ]:
ntrain = df_train.shape
display(ntrain)

各列において欠損値の割合を求める<br>
各列において欠損値の割合が0のカラムは削除

In [ ]:
df_train_nu = (df_train.isnull().sum()/len(df_train)) * 100
df_train_nu = df_train_nu.drop(df_train_nu[df_train_nu == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({"Missing Ratio" : df_train_nu})
missing_data.head(100)

In [ ]:
f, ax = plt.subplots(figsize=(15, 12))
plt.xticks(rotation="90")
sns.barplot(x=df_train_nu.index, y=df_train_nu)
plt.xlabel("Features", fontsize=15)
plt.ylabel("Percent of missing values", fontsize=15)
plt.title("Percent missing data by feature", fontsize=15)

In [ ]:
df_train.info()

In [ ]:
df_train["PoolQC"].isnull().sum()

In [ ]:
df_train["MiscFeature"].isnull().sum()

In [ ]:
df_train["Fence"].isnull().sum()

In [ ]:
df_train["FireplaceQu"].isnull().sum()

In [ ]:
df_train.drop(["PoolQC", "MiscFeature", "Alley", "Fence", "FireplaceQu"], axis=1, inplace=True)

In [ ]:
df_train.head()

In [ ]:
df_train.isnull().sum()

2回目<br>
各列でデータの欠損率を算出

In [ ]:
df_train_nu = (df_train.isnull().sum() / len(df_train)) * 100
df_train_nu = df_train_nu.drop(df_train_nu[df_train_nu==0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({"Missing Ratio" : df_train_nu})
missing_data.head(30)

In [ ]:
df_train.LotFrontage

LotFrontage：物件に面している通りの直線距離<br>
グラフの形がSalePriceと似ている

In [ ]:
ax = sns.distplot(df_train["LotFrontage"])

In [ ]:
ax = sns.boxplot(x=df_train["LotFrontage"])

LotFrontageの欠損値をデータの中央値で埋める

In [ ]:
df_train["LotFrontage"].fillna(df_train.LotFrontage.median(), inplace=True)

In [ ]:
ax = sns.distplot(df_train["LotFrontage"])

In [ ]:
ax = sns.boxplot(x=df_train["LotFrontage"])

Garage：車庫もしくは作業場

In [ ]:
df_train.GarageType

In [ ]:
df_train["GarageType"].isnull().sum()

In [ ]:
df_train["GarageType"].fillna(df_train["GarageType"].mode()[0], inplace=True)

In [ ]:
df_train["GarageType"].isnull().sum()

GarageYrBlt：GarageYearBuilt、ガレージの建築年

In [ ]:
df_train.GarageYrBlt

In [ ]:
ax = sns.distplot(df_train["GarageYrBlt"])

In [ ]:
df_train["GarageYrBlt"].isnull().sum()

In [ ]:
df_train.GarageYrBlt.median()

In [ ]:
df_train["GarageYrBlt"].fillna(df_train.GarageYrBlt.median(), inplace=True)

In [ ]:
df_train["GarageYrBlt"].isnull().sum()

In [ ]:
df_train.GarageFinish

In [ ]:
df_train["GarageFinish"].fillna(df_train.GarageFinish.mode()[0], inplace=True)

In [ ]:
df_train["GarageFinish"].isnull().sum()

In [ ]:
df_train["GarageQual"]

In [ ]:
df_train["GarageQual"].fillna(df_train.GarageQual.mode()[0], inplace=True)

In [ ]:
df_train["GarageQual"].isnull().sum()

In [ ]:
df_train.GarageCond

In [ ]:
df_train["GarageCond"].fillna(df_train.GarageCond.mode()[0], inplace=True)

In [ ]:
df_train["GarageCond"].isnull().sum()